In [1]:
import tensorflow as tf
import pickle
from PIL import Image
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import cv2
from sklearn.utils.class_weight import compute_class_weight

In [2]:
#hyper parameters:
BATCH_SIZE = 32
NUM_EPOCHS = 20

### Model Architecture

In [3]:
class CardPredictor(tf.keras.Model):
    def __init__(self, num_of_classes):
        super(CardPredictor, self).__init__()
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                                tf.keras.layers.RandomContrast(0.1),
                                tf.keras.layers.GaussianNoise(0.1),
                                tf.keras.layers.RandomBrightness(0.1),
                                tf.keras.layers.RandomTranslation(0.1, 0.1),
                            ])
        
        self.architecture = [        
                tf.keras.layers.InputLayer((300, 300, 3)),
                data_augmentation,

                tf.keras.layers.Conv2D(64, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(128, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(256, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(512, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(1024, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.GlobalAveragePooling2D(),                                   # Flatten to vector
                
                tf.keras.layers.Dense(256, activation='relu'),               # Fully connected layer
                tf.keras.layers.Dropout(0.5),                                # Prevent overfitting
                tf.keras.layers.Dense(num_of_classes, activation='softmax')
                ]
        
        
        self.sequential = tf.keras.Sequential(self.architecture, name="card_predictor_classes_"+ str(num_of_classes))
        
    def call(self, x):
        """ Passes input image through the network. """
        return self.sequential(x)

### Dataset Loading

In [4]:
#TRAIN
with open('train.pkl', 'rb') as file:
    train_data = pickle.load(file)
    
train_suit_data = {}
train_rank_data = {}

for key, inner_dict in train_data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['concept_label']
    rank = value[:13]
    suit = value[13:]
    train_rank_data[img_path] = rank
    train_suit_data[img_path] = suit
    
with open('val.pkl', 'rb') as file:
    test_data = pickle.load(file)

#TEST
test_suit_data = {}
test_rank_data = {}

for key, inner_dict in test_data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['concept_label']
    rank = value[:13]
    suit = value[13:]
    test_rank_data[img_path] = rank
    test_suit_data[img_path] = suit

In [9]:
# Define the image loading and preprocessing function
def load_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Preprocess the dataset for training
train_rank_dataset = (
    tf.data.Dataset.from_tensor_slices((list(train_rank_data.keys()), list(train_rank_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
train_suit_dataset = (
    tf.data.Dataset.from_tensor_slices((list(train_suit_data.keys()), list(train_suit_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
test_rank_dataset = (
    tf.data.Dataset.from_tensor_slices((list(test_rank_data.keys()), list(test_rank_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
test_suit_dataset = (
    tf.data.Dataset.from_tensor_slices((list(test_suit_data.keys()), list(test_suit_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

half_size = int(len(test_rank_data)*0.5)
val_rank_dataset = test_rank_dataset.take(half_size)
test_rank_dataset = test_rank_dataset.skip(half_size)
val_suit_dataset = test_suit_dataset.take(half_size)
test_suit_dataset = test_suit_dataset.skip(half_size)

### Model Initialization

In [5]:
suit_model = CardPredictor(4)
suit_model.build((None, 300, 300, 3))
suit_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
rank_model = CardPredictor(13)
rank_model.build((None, 300, 300, 3))
rank_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

2025-05-04 13:54:25.457101: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2025-05-04 13:54:25.457137: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-05-04 13:54:25.457143: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-05-04 13:54:25.457162: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-04 13:54:25.457178: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Model Training

In [63]:
suit_model.fit(train_suit_dataset, 
                validation_data=val_suit_dataset, 
                epochs=NUM_EPOCHS, 
                verbose=1,
                callbacks=[
                    tf.keras.callbacks.ModelCheckpoint(
                        filepath='suit_model_ckpt.keras',
                        save_best_only=True,
                    )
                ])

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 95s 391ms/step - accuracy: 0.9580 - loss: 0.1467 - val_accuracy: 0.9867 - val_loss: 0.0653
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 98s 404ms/step - accuracy: 0.9747 - loss: 0.1042 - val_accuracy: 0.9937 - val_loss: 0.0652
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 99s 406ms/step - accuracy: 0.9843 - loss: 0.0702 - val_accuracy: 0.9970 - val_loss: 0.0265
Epoch 4/20
132/219 ━━━━━━━━━━━━━━━━━━━━ 30s 355ms/step - accuracy: 0.9959 - loss: 0.0163

KeyboardInterrupt: 

In [13]:
rank_model.fit(train_rank_dataset, 
                validation_data=val_rank_dataset, 
                epochs=NUM_EPOCHS, 
                verbose=1,
                callbacks=[
                    tf.keras.callbacks.ModelCheckpoint(
                        filepath='rank_model_ckpt.keras',
                        save_best_only=True,
                    )
                ])

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 98s 408ms/step - accuracy: 0.0827 - loss: 3.0769 - val_accuracy: 0.0806 - val_loss: 2.5680
Epoch 2/20
 90/219 ━━━━━━━━━━━━━━━━━━━━ 44s 347ms/step - accuracy: 0.0679 - loss: 2.5710

KeyboardInterrupt: 

### Model Load From CheckPoint

In [16]:
suit_model.load_weights("suit_model_ckpt.keras")

In [14]:
rank_model.load_weights("rank_model_ckpt.keras")

### Model Evaluation

In [17]:
suit_model.evaluate(test_suit_dataset, verbose=1)

2025-05-04 14:01:06.045513: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/card_predictor_1/card_predictor_classes_4_1/sequential_1/random_rotation_1/Tan/_82]]
2025-05-04 14:01:06.045548: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 11564020271169716519
2025-05-04 14:01:06.045554: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 6563476843256586929
2025-05-04 14:01:06.045564: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2571453187023012763
2025-05-04 14:01:06.045569: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8036324582959145725
2025-05-04 14:01:06.045579: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv ite

ValueError: math domain error

In [15]:
rank_model.evaluate(test_rank_dataset, verbose=1)

2025-05-04 14:00:39.295486: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2723440438872448321
2025-05-04 14:00:39.295503: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14573204067390775883
2025-05-04 14:00:39.295531: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1278585687257851537
2025-05-04 14:00:39.295544: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1171949211444734365
2025-05-04 14:00:39.295548: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 12622905231823637853
2025-05-04 14:00:39.295553: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 7265161825486515713
2025-05-04 14:00:39.295557: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv i

ValueError: math domain error